In [ ]:
import torch
import types
import argparse
import os
import torch.distributed as dist
import numpy as np
import time
import random
import tarfile
import itertools
import yaml
import wandb

In [ ]:
# toolkit
import gTDR.utils.EvolveGCN as utils
from gTDR.trainers.EvolveGCN_trainer import Trainer

## Arguments & Parameters

Specify the setup in config, including:
* `folder`: (str) The path of the dataset.
* `use_cuda`: (bool) Whether to use CUDA for GPU acceleration.
* `use_logfile`: (bool) If true, we save the output in a log file, if false the result is in stdout.
* `save_results`: (bool) Whether to save the training and testing results.
* `save_path`: (str) The path where to save the trained model and results.
* `seed`: (int) The random seed for reproducibility.

In [ ]:
config_filename = "../configs/EvolveGCN_O_sbm50_parameters.yaml"
with open(config_filename) as f:
    configs = yaml.load(f, Loader=yaml.SafeLoader)
args = types.SimpleNamespace(**configs)

Use GPU.

In [ ]:
args.use_cuda = (torch.cuda.is_available() and args.use_cuda)
args.device='cpu'
if args.use_cuda:
    args.device='cuda'
print ("use CUDA:", args.use_cuda, "- device:", args.device)

Set seed for reproducibility.

In [ ]:
seed = args.seed
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

Complete the specification of `args`.

In [ ]:
args = utils.build_random_hyper_params(args)

Start `wandb` for monitoring experiment (train loss, validation loss, and `target_measure` specified in config). See the config file for choices of `target_measure`. 

In [ ]:
run = wandb.init(project="EvolveGCN_O", name="sbm50")

## Data (Part 1)

In this demo, we use the `sbm50` dataset.

**First, unzip dataset located at [../data/sbm50/](../data/sbm50/):**

`tar -xvzf ../data/sbm50/sbm_50t_1000n_adj.csv.tar.gz -C ../data/sbm50/`

## Data (Part 2)

Next, define a dataset class that contains class memebers `node_feats` and `edges` for temporal link prediction.

Inside this class, the unzipped file `sbm_50t_1000n_adj.csv` is read to populate the class members.

* `node_feats` is a tensor of node features.
* `edges` is a dictionary with `idx` and `vals` as keys.

In [ ]:
class sbm_dataset():
    def __init__(self, args):
        assert args.task in ['link_pred'], 'sbm only implements link_pred'
        self.ecols = utils.Namespace({'FromNodeId': 0,
                                      'ToNodeId': 1,
                                      'Weight': 2,
                                      'TimeStep': 3
                                     })
        args.sbm_args = utils.Namespace(args.sbm_args)

        #build edge data structure
        edges = self.load_edges(args.sbm_args)
        timesteps = utils.aggregate_by_time(edges[:,self.ecols.TimeStep], args.sbm_args.aggr_time)
        self.max_time = timesteps.max()
        self.min_time = timesteps.min()
        print ('TIME', self.max_time, self.min_time )
        edges[:,self.ecols.TimeStep] = timesteps

        edges[:,self.ecols.Weight] = self.cluster_negs_and_positives(edges[:,self.ecols.Weight])
        self.num_classes = edges[:,self.ecols.Weight].unique().size(0)

        self.edges = self.edges_to_sp_dict(edges)
        
        #random node features
        self.num_nodes = int(self.get_num_nodes(edges))
        self.feats_per_node = args.sbm_args.feats_per_node
        self.nodes_feats = torch.rand((self.num_nodes,self.feats_per_node))

        self.num_non_existing = self.num_nodes ** 2 - edges.size(0)

    def cluster_negs_and_positives(self,ratings):
        pos_indices = ratings >= 0
        neg_indices = ratings < 0
        ratings[pos_indices] = 1
        ratings[neg_indices] = 0
        return ratings

    def prepare_node_feats(self,node_feats):
        node_feats = node_feats[0]
        return node_feats

    def edges_to_sp_dict(self,edges):
        idx = edges[:,[self.ecols.FromNodeId,
                       self.ecols.ToNodeId,
                       self.ecols.TimeStep]]

        vals = edges[:,self.ecols.Weight]
        return {'idx': idx,
                'vals': vals}

    def get_num_nodes(self,edges):
        all_ids = edges[:,[self.ecols.FromNodeId,self.ecols.ToNodeId]]
        num_nodes = all_ids.max() + 1
        return num_nodes

    def load_edges(self,sbm_args, starting_line = 1):
        file = os.path.join(sbm_args.folder,sbm_args.edges_file)
        with open(file) as f:
            lines = f.read().splitlines()
        edges = [[float(r) for r in row.split(',')] for row in lines[starting_line:]]
        edges = torch.tensor(edges,dtype = torch.long)
        return edges

    def make_contigous_node_ids(self,edges):
        new_edges = edges[:,[self.ecols.FromNodeId,self.ecols.ToNodeId]]
        _, new_edges = new_edges.unique(return_inverse=True)
        edges[:,[self.ecols.FromNodeId,self.ecols.ToNodeId]] = new_edges
        return edges

Create the dataset.

In [ ]:
args.sbm_args = args.sbm50_args
dataset = sbm_dataset(args)

## Model

Build model. In this demo, `args.model=egcn_o`.

In [ ]:
model = utils.build_model(args, dataset, task='link_prediction')

## Training

You may specify these training parameters in config:

* `train_proportion`: (float) The proportion of the dataset used for training. 

* `dev_proportion`: (float) The proportion of the dataset used for validation.

* `num_epochs`: (int) The number of epochs to train the model.

* `steps_accum_gradients`: (int) The number of steps to accumulate gradients before updating the model parameters. 

* `learning_rate`: (float) The learning rate for the Adam optimizer.

* `early_stop_patience`: (int) The number of epochs with no improvement after which training will be stopped. 

* `adj_mat_time_window`: (int) The time window to create the adjacency matrix for each time step. This parameter is not used directly in the trainer but it might be used in some other parts of the code.

* `data_loading_params` 
    * `batch_size`: (int) number of data samples propagated through the network at once. 
    * `num_workers`: (int) number of subprocesses to use for data loading. The main benefit of using multiple processes is that they can use separate memory and CPUs to load data in parallel. 

In [ ]:
trainer = Trainer(args, model=model)
trainer.train(use_wandb=True)

## Inference

Load the best check point and perform testing.

In [ ]:
trainer.load_best_checkpoint()
trainer.test()

In [ ]:
wandb.finish()